# Convert between Lat/Long/Height and ECEF XYZ



In [11]:
include("../modules/geometry.jl")

#example Lat/Long/Height
geo=[35.38987,-111.8116,9748.89523]
println("GEO: ", geo)

#compute ECEF XYZ based LLH
xyz = Geometry.geo_to_xyz(geo)
println("GEO -> XYZ: ", xyz)

#convert XYZ back to LLH
geo = Geometry.xyz_to_geo(xyz)
println("GEO -> XYZ -> GEO", geo)

#convert lots of LLH points to XYZ
@time xyz = Geometry.geo_to_xyz(repeat(geo, 1, 1000))

#convert lots of XYZ points to LLH
@time geo = Geometry.xyz_to_geo(xyz)


GEO: [35.38987, -111.8116, 9748.89523]


GEO -> XYZ: [-1.9370856627101207e6, -4.840217451858883e6, 3.6788596091408627e6]
GEO -> XYZ -> GEO[35.3898700000064, -111.81160000000001, 9748.895230504684]
  0.102775 seconds (361.41 k allocations: 16.774 MiB)
  0.133940 seconds (332.47 k allocations: 14.778 MiB, 5.74% gc time)


3×1000 Array{Float64,2}:
   35.3899    35.3899    35.3899  …    35.3899    35.3899    35.3899
 -111.812   -111.812   -111.812      -111.812   -111.812   -111.812
 9748.9     9748.9     9748.9        9748.9     9748.9     9748.9

# Convert SCH coordinates to Approximate ECEF XYZ 

In [16]:
#create PEG point based on LLH coordinates (peg point is origin of SCH CS)
peg = Geometry.PegPoint(geo[1], geo[2], geo[3])

#example SCH coordinate
sch=[-19766.4,23.145535442,9748.895229822]

#convert SCH coordinate to XYZ
xyz = Geometry.sch_to_xyz(sch, peg)
println("SCH -> XYZ", xyz)

#convert lots of SCH to XYZ. This would be how one would convert an sch grid to an XYZ grid
@time xyz = Geometry.sch_to_xyz(repeat(sch,1, 1000), peg)

SCH -> XYZ[-1.9497034352294554e6; -4.845945391150809e6; 3.664721314257136e6]
  0.000308 seconds (4.00 k allocations: 484.531 KiB)


3×1000 Array{Float64,2}:
 -1.9497e6   -1.9497e6   -1.9497e6   …  -1.9497e6   -1.9497e6   -1.9497e6
 -4.84595e6  -4.84595e6  -4.84595e6     -4.84595e6  -4.84595e6  -4.84595e6
  3.66472e6   3.66472e6   3.66472e6      3.66472e6   3.66472e6   3.66472e6

# Working with Quaternions for vector and frame rotations

In [19]:
q = Geometry.quat(45, [0,1,0]) #create a quaternion to rotate a vector by 45 degrees about yaxis [0,1,0]
rotated_vec = Geometry.rotate_vec([1,0,0], q) #rotate a vector aligned with the x-axis, by q
println("Rotated Vector: ", rotated_vec)


# combine rotations q3*q2*q1 means first q1 then q2 then q3
q1 = Geometry.quat(90, [1,0,0]) #create a quaternion to rotate a vector by 90 degrees about xaxis
q2 = Geometry.quat(90, [0,1,0]) #create a quaternion to rotate a vector by 90 degrees about yaxis
q3 = Geometry.quat(90, [0,0,1]) #create a quaternion to rotate a vector by 90 degrees about zaxis
rotated_vec = Geometry.rotate_vec([1,0,0], q1) #rotate a vector aligned with the x-axis, by q1
println("\nRotated Vector: ", rotated_vec)
rotated_vec = Geometry.rotate_vec([1,0,0], q2) #rotate a vector aligned with the x-axis, by q2
println("Rotated Vector: ", rotated_vec)
rotated_vec = Geometry.rotate_vec([1,0,0], q3) #rotate a vector aligned with the x-axis, by q3
println("Rotated Vector: ", rotated_vec)
rotated_vec = Geometry.rotate_vec([1,0,0], q2*q1) #rotate a vector aligned with the x-axis, by q2*q1
println("Rotated Vector (q2*q1): ", rotated_vec)
rotated_vec = Geometry.rotate_vec([1,0,0], q1*q2) #rotate a vector aligned with the x-axis, by q1*q2
println("Rotated Vector (q1*q2): ", rotated_vec)
rotated_vec = Geometry.rotate_vec([1,0,0], q1*q2*q3) #rotate a vector aligned with the x-axis, by q1*q2*q3
println("Combined Rotated Vector, q1*q2*q3: ", rotated_vec)
rotated_vec = Geometry.rotate_vec([1,0,0], q1*q3*q2) #rotate a vector aligned with the x-axis, by q1*q3*q2
println("Combined Rotated Vector, q1*q3*q2: ", rotated_vec)
rotated_vec = Geometry.rotate_vec([1,0,0], q2*q1*q3) #rotate a vector aligned with the x-axis, by q2*q1*q3
println("Combined Rotated Vector, q2*q1*q3: ", rotated_vec)
rotated_vec = Geometry.rotate_vec([1,0,0], q2*q3*q1) #rotate a vector aligned with the x-axis, by q2*q3*q1
println("Combined Rotated Vector, q2*q3*q1: ", rotated_vec)
rotated_vec = Geometry.rotate_vec([1,0,0], q3*q2*q1) #rotate a vector aligned with the x-axis, by q3*q2*q1
println("Combined Rotated Vector, q3*q2*q1: ", rotated_vec)
rotated_vec = Geometry.rotate_vec([1,0,0], q3*q1*q2) #rotate a vector aligned with the x-axis, by q3*q1*q2
println("Combined Rotated Vector, q3*q1*q2: ", rotated_vec)

# rotate a frame, then describe vector in the rotated frame
qy = Geometry.quat(90, [0,1,0]) #create a quaternion to rotate a frame by 90 degrees about yaxis [0,1,0]
qz= Geometry.quat(90, [0,0,1]) #create a quaternion to rotate a frame by 90deg about z-axis
projected_vec = Geometry.rotate_frame([1,0,0], qy) #project a vector aligned with the x-axis, to a rotated frame described by q
println("\nProjected Vector: ", projected_vec)
projected_vec = Geometry.rotate_frame([1,0,0], qz) #project a vector aligned with the x-axis, to a rotated frame described by q
println("Projected Vector: ", projected_vec)
# EXtrinsic vs. intrinsic rotations:
# intrinsic rotation: second rotation is about rotated frame
# extrinsic rotation: second rotation is about the original frmae
# q1*q2 can be thought of in two ways:
# (A) rotate q1 and then rotate q2 about rotated frame (intrinsic)
# (B) rotate q2 first and then rotate q1 about original frame (extrinsic)
projected_vec = Geometry.rotate_frame([1,0,0], qy*qz) #project a vector aligned with the x-axis, to a rotated frame described by q
println("\nIntrinsic projected vector: ", projected_vec)
projected_vec = Geometry.rotate_frame([1,0,0], qz*qy) #project a vector aligned with the x-axis, to a rotated frame described by q
println("Extrinsic projected vector: ", projected_vec)


Rotated Vector: [0.7071067811865475, 0.0, -0.7071067811865476]

Rotated Vector: [1.0, 0.0, 0.0]
Rotated Vector: [0.0, 0.0, -1.0]
Rotated Vector: [0.0, 1.0, 0.0]
Rotated Vector (q2*q1): [0.0, 0.0, -1.0]
Rotated Vector (q1*q2): [0.0, 1.0, 0.0]
Combined Rotated Vector, q1*q2*q3: [0.0, 0.0, 1.0]
Combined Rotated Vector, q1*q3*q2: [0.0, 1.0, 0.0]
Combined Rotated Vector, q2*q1*q3: [1.0, 0.0, 0.0]
Combined Rotated Vector, q2*q3*q1: [0.0, 1.0, 0.0]
Combined Rotated Vector, q3*q2*q1: [0.0, 0.0, -1.0]
Combined Rotated Vector, q3*q1*q2: [-1.0, 0.0, 0.0]

Projected Vector: [0.0, 0.0, 1.0]
Projected Vector: [0.0, -1.0, 0.0]

Intrinsic projected vector: [0.0, 0.0, 1.0]
Extrinsic projected vector: [0.0, -1.0, 0.0]
